In [ ]:
!pip install -q /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.1-py3-none-any.whl

In [ ]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split, KFold
from pytorch_tabnet.tab_model import TabNetClassifier
import janestreet
import cudf
import seaborn as sns
import os

In [ ]:
train_df = cudf.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')
train_df = train_df.to_pandas()
train_df = train_df.query('weight != 0')

In [ ]:
train_df['action'] = ((train_df['weight'].values * train_df['resp'].values) > 0).astype('int')
train_df = train_df.fillna(-999)

features = [c for c in train_df.columns if 'feature' in c]

X = train_df.loc[:, features].values
y = train_df.loc[:, 'action'].values

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.10,shuffle=False)

In [ ]:
n_d = n_a = 24

clf = TabNetClassifier(n_d=n_d,n_a=n_a)

max_epochs = 20

clf.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],
    max_epochs=max_epochs,
    patience=5,
    batch_size=1024, virtual_batch_size=512,
    num_workers=4,
    drop_last=False
)


In [ ]:
%timeit
env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set

for (test_df, sample_prediction_df) in iter_test:
    test_df = test_df.fillna(-999)
    X_test = test_df.loc[:, features].values
    preds = clf.predict(X_test)  
    action = preds.astype('int')

    sample_prediction_df.action = action
    env.predict(sample_prediction_df)